# Classification Mesh 


Le MeSH (Medical Subject Headings) est un vocabulaire contrôlé utilisé pour organiser et indexer les articles médicaux. Il est structuré de manière hiérarchique en “arbres” (MeSH trees), où chaque concept est associé à un code unique appelé numéro d’arbre.

“Catégorie principale des Maladies nutritionnelles et métaboliques” est ainsi associé au code C14, où “C” représente la catégorie des maladies et “14” spécifie le groupe des maladies nutritionnelles et métaboliques. Cette organisation facilite la navigation du général au spécifique, aidant ainsi les chercheurs à trouver des informations précises.

Mais il est aussi d'être plus precis en selectionnant des maladies plus loin dans l'**arborescences**.

Par le Diabete de type 1 correspond au code MeSH C18.452.394.750.124 qui se décompose comme suit :

*   C : Catégorie des maladies
*   C18 : Catégorie principale des Maladies nutritionnelles et métaboliques.
*   C18.452 : Sous-catégorie des Maladies métaboliques.
*   C18.452.394 : Sous-sous-catégorie des Troubles du métabolisme du glucose.
*   C18.452.394.750 : Sous-sous-sous-catégorie du Diabète sucré.
*   C18.452.394.750.124 : Terme spécifique pour le Diabète sucré de type 1.

Cette hiérarchie reflète l’organisation des termes MeSH, permettant de situer chaque condition médicale dans un contexte plus large et de naviguer efficacement dans les informations médicales.

In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
import os

# Ajouter le dossier courant au PATH si ce n'est pas déjà fait
sys.path.append(os.path.abspath("."))

import MeSH
print(MeSH.Hello())

Hello from MeSH!


Le dictionnaire est disponible au format XML à cet [URL](https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/?_gl=1*1vhz8wv*_ga*MTkwNTMyNjEyOC4xNzM2Njg2Mzcy*_ga_7147EPK006*MTczNzEwNTQ1MS4xMi4xLjE3MzcxMDU4OTMuMC4wLjA.*_ga_P1FPTH9PL4*MTczNzEwNTQ1MS4xNC4xLjE3MzcxMDU4OTMuMC4wLjA)

In [3]:
import requests

# URL et chemin local du fichier XML
url_fichier = "https://nlmpubs.nlm.nih.gov/projects/mesh/MESH_FILES/xmlmesh/desc2025.xml"
chemin_fichier = "data/desc2025.xml"  # Nom du fichier local

# Vérifier si le fichier existe déjà
if not os.path.exists(chemin_fichier):
    try:
        response = requests.get(url_fichier)
        response.raise_for_status()  # Vérifie si la requête a réussi

        with open(chemin_fichier, "wb") as fichier:
            fichier.write(response.content)

        print(f"Fichier téléchargé avec succès : {chemin_fichier}")
    except requests.RequestException as e:
        print(f"Erreur lors du téléchargement du fichier XML : {e}")
        exit(1)  # Arrête l'exécution en cas d'échec
else:
    print(f"Le fichier {chemin_fichier} existe déjà. Pas de téléchargement nécessaire.")

# Analyse du fichier téléchargé
racine = MeSH.chargement_arbre_mesh(chemin_fichier)

Le fichier data/desc2025.xml existe déjà. Pas de téléchargement nécessaire.
Fichier XML analysé avec succès : data/desc2025.xml


## Chargement données

In [4]:
df = pd.read_parquet('data/df_final')
df.head(3)

,pmcid,title,gender,case_text,keywords,major_mesh_terms,mesh_terms,journal,doi
0,PMC5287946,Malignant adenohypophysis spindle cell oncocyt...,Male,A 30-year-old man came to Peking Union Medical...,None,"[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...","[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...",Medicine (Baltimore),10.1097/MD.0000000000005657
1,PMC3678730,Misclassification of pregnancy-related deaths ...,Female,"For each death, the DSS has systematically rec...",None,"[Cause of Death, Data Collection / standards, ...","[Cause of Death, Data Collection / standards, ...",Trop Med Int Health,10.1111/tmi.12012
2,PMC7201103,Gender Differences in Patients With COVID-19: ...,Female,The demographic and clinical characteristics a...,"[covid-19, sars, sars-cov-2, female, gender, m...","[COVID-19 / epidemiology, COVID-19 / mortality...","[COVID-19 / epidemiology, COVID-19 / mortality...",Front Public Health,10.3389/fpubh.2020.00152


In [5]:
print(type(df.major_mesh_terms[4]))
print(type(MeSH.list_to_dict(df.major_mesh_terms[4])))
print(MeSH.list_to_dict(df.major_mesh_terms[4]))

<class 'numpy.ndarray'>
<class 'dict'>
{'therapy': {'Arteriovenous Fistula', 'Intracranial Arteriovenous Malformations'}, 'pathology': {'Carotid Artery, Internal', 'Cavernous Sinus'}, 'complications': {'Ehlers-Danlos Syndrome'}, 'methods': {'Embolization, Therapeutic', 'Endovascular Procedures'}}


In [6]:
df['dict_mesh'] = df.major_mesh_terms.apply(MeSH.list_to_dict)
df.head()

,pmcid,title,gender,case_text,keywords,major_mesh_terms,mesh_terms,journal,doi,dict_mesh
0,PMC5287946,Malignant adenohypophysis spindle cell oncocyt...,Male,A 30-year-old man came to Peking Union Medical...,None,"[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...","[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...",Medicine (Baltimore),10.1097/MD.0000000000005657,"{'pathology': {'Pituitary Neoplasms', 'Adenoma..."
1,PMC3678730,Misclassification of pregnancy-related deaths ...,Female,"For each death, the DSS has systematically rec...",None,"[Cause of Death, Data Collection / standards, ...","[Cause of Death, Data Collection / standards, ...",Trop Med Int Health,10.1111/tmi.12012,"{'No Qualifier': {'Mortality', 'Cause of Death..."
2,PMC7201103,Gender Differences in Patients With COVID-19: ...,Female,The demographic and clinical characteristics a...,"[covid-19, sars, sars-cov-2, female, gender, m...","[COVID-19 / epidemiology, COVID-19 / mortality...","[COVID-19 / epidemiology, COVID-19 / mortality...",Front Public Health,10.3389/fpubh.2020.00152,{'epidemiology': {'Severe Acute Respiratory Sy...
3,PMC4108191,Acute pulmonary embolism caused by enlarged ut...,Female,"A 42-year-old, non-smoking, virgin woman prese...","[echocardiography, leiomyoma, pulmonary emboli...","[Pulmonary Embolism / etiology, Uterine Neopla...","[Pulmonary Embolism / etiology, Uterine Neopla...",Am J Case Rep,10.12659/AJCR.890607,"{'etiology': {'Pulmonary Embolism'}, 'complica..."
4,PMC4508704,Transvenous embolization for carotid-cavernous...,Female,The patient was a 37-year-old woman with a his...,None,"[Arteriovenous Fistula / therapy, Carotid Arte...","[Arteriovenous Fistula / therapy, Carotid Arte...",Neurol Med Chir (Tokyo),10.2176/nmc.cr.2013-0007,"{'therapy': {'Arteriovenous Fistula', 'Intracr..."


## Extraction Code MeSH

In [7]:
from tqdm import tqdm
tqdm.pandas()  # Pour que apply() affiche une barre de progression

In [8]:
from functools import lru_cache

@lru_cache(maxsize=None)
def rechercher_descripteur_cached(terme, racine):
    return MeSH.recherche_descripteur(racine, terme, format_reponse='numero_mesh')


def extraire_codes_disease_C_cached(dict_mesh_ligne, racine):
    """
    Version optimisée avec cache + rapide + propre.
    """
    codes = set()

    if not isinstance(dict_mesh_ligne, dict):
        return []

    for termes in dict_mesh_ligne.values():
        for terme in termes:
            list_mesh = rechercher_descripteur_cached(terme, racine)
            if list_mesh:
                for item in list_mesh:
                    if item.startswith("C") and len(item) > 2:
                        code = item[1:3]
                        if code.isdigit():
                            codes.add(code)

    return sorted(codes)

In [26]:
df2 = df.head(500).copy(deep=True)
df2

,pmcid,title,gender,case_text,keywords,major_mesh_terms,mesh_terms,journal,doi,dict_mesh
0,PMC5287946,Malignant adenohypophysis spindle cell oncocyt...,Male,A 30-year-old man came to Peking Union Medical...,None,"[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...","[Adenoma, Oxyphilic / pathology, Ki-67 Antigen...",Medicine (Baltimore),10.1097/MD.0000000000005657,"{'pathology': {'Pituitary Neoplasms', 'Adenoma..."
1,PMC3678730,Misclassification of pregnancy-related deaths ...,Female,"For each death, the DSS has systematically rec...",None,"[Cause of Death, Data Collection / standards, ...","[Cause of Death, Data Collection / standards, ...",Trop Med Int Health,10.1111/tmi.12012,"{'No Qualifier': {'Mortality', 'Cause of Death..."
2,PMC7201103,Gender Differences in Patients With COVID-19: ...,Female,The demographic and clinical characteristics a...,"[covid-19, sars, sars-cov-2, female, gender, m...","[COVID-19 / epidemiology, COVID-19 / mortality...","[COVID-19 / epidemiology, COVID-19 / mortality...",Front Public Health,10.3389/fpubh.2020.00152,{'epidemiology': {'Severe Acute Respiratory Sy...
3,PMC4108191,Acute pulmonary embolism caused by enlarged ut...,Female,"A 42-year-old, non-smoking, virgin woman prese...","[echocardiography, leiomyoma, pulmonary emboli...","[Pulmonary Embolism / etiology, Uterine Neopla...","[Pulmonary Embolism / etiology, Uterine Neopla...",Am J Case Rep,10.12659/AJCR.890607,"{'etiology': {'Pulmonary Embolism'}, 'complica..."
4,PMC4508704,Transvenous embolization for carotid-cavernous...,Female,The patient was a 37-year-old woman with a his...,None,"[Arteriovenous Fistula / therapy, Carotid Arte...","[Arteriovenous Fistula / therapy, Carotid Arte...",Neurol Med Chir (Tokyo),10.2176/nmc.cr.2013-0007,"{'therapy': {'Arteriovenous Fistula', 'Intracr..."
...,...,...,...,...,...,...,...,...,...,...
495,PMC4293555,Rhupus syndrome,Female,A 45-year-old female presented with a facial r...,None,"[Arthritis, Rheumatoid / complications, Lupus ...","[Arthritis, Rheumatoid / complications, Lupus ...",Korean J Intern Med,10.3904/kjim.2015.30.1.131,"{'complications': {'Lupus Erythematosus, Syste..."
496,PMC3892021,Acute hydrocephalus caused by intraspinal neur...,Male,A 64-year-old man was admitted to our institut...,None,"[Hydrocephalus / etiology, Neurocysticercosis ...","[Hydrocephalus / etiology, Neurocysticercosis ...",BMC Res Notes,10.1186/1756-0500-7-2,"{'etiology': {'Hydrocephalus'}, 'complications..."
497,PMC6522167,Fast track extubation in paediatric cardiothor...,Female,During humanitarian cardiothoracic surgical mi...,"[congenital heart disease, fast-tracking, mech...","[Airway Extubation / methods, Anesthetics / ad...","[Airway Extubation / methods, Anesthetics / ad...",Pan Afr Med J,10.11604/pamj.2019.32.55.14019,"{'methods': {'Airway Extubation', 'Cardiac Sur..."
498,PMC3708325,Huge mucinous cystadenoma of the pancreas mist...,Female,A 50 year old lady presented with four months ...,"[cystic tumor, cystadenoma, pancreas, pseudocyst]","[Cystadenoma, Mucinous / diagnosis, Pancreatic...","[Cystadenoma, Mucinous / diagnosis, Pancreatic...",Pan Afr Med J,10.11604/pamj.2013.15.6.2494,"{'diagnosis': {'Cystadenoma, Mucinous', 'Pancr..."


In [29]:
# Application ligne par ligne avec racine et objet MeSH passés comme arguments
df2["codes_mesh_extraits"] = df2["dict_mesh"].progress_apply(lambda x: extraire_codes_disease_C_cached(x, racine))

100%|██████████| 500/500 [00:00<00:00, 96336.63it/s]


In [37]:
df2["target"] = df2["codes_mesh_extraits"].apply(lambda x: vectorizer(x))

In [39]:

df2.to_csv("data/df_sample_500")

***

In [31]:
import numpy as np

def vectorizer(indices_positifs, taille_vecteur=26):
    """
    Crée un vecteur binaire avec des 1 aux indices spécifiés et 0 ailleurs.
    Retourne None si la liste d'entrée est vide.
    
    Parameters:
    - indices_positifs (list[int]): Liste des indices à mettre à 1 (indices de 1 à taille_vecteur).
    - taille_vecteur (int): Taille du vecteur résultat (défaut=26).
    
    Returns:
    - numpy.ndarray | None : vecteur binaire ou None si indices_positifs vide.
    """
    indices_positifs = list(map(int, indices_positifs))
    # Retourne None si la liste est vide
    if not indices_positifs:
        return None
    
    # Vérification des entrées
    if not all(1 <= idx <= taille_vecteur for idx in indices_positifs):
        raise ValueError(f"Tous les indices doivent être compris entre 1 et {taille_vecteur}.")
    
    # Création du vecteur cible avec des zéros
    vecteur_binaire = np.zeros(taille_vecteur, dtype=int)
    
    # Attribution des 1 aux positions spécifiées (ajustement des indices à partir de 1)
    vecteur_binaire[np.array(indices_positifs) - 1] = 1
    
    return np.vstack(vecteur_binaire)

In [36]:
vectorizer(['10'])

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]])

# Exemples / Archives

In [12]:
def extraire_codes_mesh(dict_mesh_ligne, racine, mesh_obj):
    """
    Extrait les deux chiffres suivant le 'C' des codes MeSH pour les maladies (catégorie C).

    Args:
        dict_mesh_ligne (dict): Dictionnaire {qualificateur: set(termes)}
        racine (str): Racine MeSH utilisée pour la recherche.
        mesh_obj: Objet MeSH avec la méthode recherche_descripteur(term, racine, format_reponse='numero_mesh')

    Returns:
        list: Liste triée des codes 'C' (2 chiffres après le 'C'), sans doublons.
    """
    set_final = set()

    if not isinstance(dict_mesh_ligne, dict):
        return []

    for mesh_terms in dict_mesh_ligne.values():
        if not isinstance(mesh_terms, (set, list)):
            continue

        for term in mesh_terms:
            try:
                list_mesh = mesh_obj.recherche_descripteur(racine, str(term), format_reponse='numero_mesh')
                if not list_mesh:
                    continue

                for item in list_mesh:
                    if isinstance(item, str) and item.startswith('C') and len(item) >= 3:
                        code = item[1:3]  # Prend les deux chiffres après le 'C'
                        if code.isdigit():
                            set_final.add(code)
            except Exception as e:
                print(f"Erreur avec le terme '{term}': {e}")
                continue

    return sorted(set_final)

In [13]:
df["dict_mesh"].head(100)

0     {'pathology': {'Pituitary Neoplasms', 'Adenoma...
1     {'No Qualifier': {'Mortality', 'Cause of Death...
2     {'epidemiology': {'Severe Acute Respiratory Sy...
3     {'etiology': {'Pulmonary Embolism'}, 'complica...
4     {'therapy': {'Arteriovenous Fistula', 'Intracr...
                            ...                        
95    {'genetics': {'Spasms, Infantile', 'Calcium Ch...
96    {'surgery': {'Acetabulum', 'Femoracetabular Im...
97    {'pathology': {'Iris', 'Choroid'}, 'diagnosis'...
98    {'genetics': {'cis-trans-Isomerases', 'Leber C...
99    {'pathology': {'Lymphangioma', 'Pelvic Neoplas...
Name: dict_mesh, Length: 100, dtype: object

In [14]:
df2 = pd.DataFrame()
# Application ligne par ligne avec racine et objet MeSH passés comme arguments
df2["codes_mesh_extraits"] = df["dict_mesh"].head(100).apply(lambda x: extraire_codes_mesh(x, racine, MeSH))

In [15]:
from collections import OrderedDict

i = 4
set_final = set()

#rajouter fonction qui prend en arg un dict

for key in df.dict_mesh[i].keys():
    mesh_terms = df.dict_mesh[i][key]  # C'est un set
    print("\nQualificateur :", key)

    for term in mesh_terms:  # Itérer sur le set (même si un seul élément)
        list_mesh = MeSH.recherche_descripteur(racine, str(term), format_reponse='numero_mesh')
        
        # Filtrer, extraire et ajouter les éléments au set
        set_final.update(item.split('.')[0][-2:] for item in list_mesh if item.startswith('C'))
    
    print(sorted(set_final))  # Affichage trié

# Conversion finale en liste triée
result = sorted(set_final)

print("\nListe finale triée :", result)




Qualificateur : therapy
['10', '14', '16', '23']

Qualificateur : pathology
['10', '14', '16', '23']

Qualificateur : complications
['10', '14', '15', '16', '17', '23']

Qualificateur : methods
['10', '14', '15', '16', '17', '23']

Liste finale triée : ['10', '14', '15', '16', '17', '23']


In [16]:
from collections import OrderedDict

i = 4
set_final = set()

#rajouter fonction qui prend en arg un dict

for key in df.dict_mesh[i].keys():
    mesh_terms = df.dict_mesh[i][key]  # C'est un set
    print("\nQualificateur :", key)

    for term in mesh_terms:  # Itérer sur le set (même si un seul élément)
        list_mesh = MeSH.recherche_descripteur(racine, str(term), format_reponse='numero_mesh')
        
        # Filtrer, extraire et ajouter les éléments au set
        set_final.update(MeSH.categorie_haute(racine, item, level=1) for item in list_mesh if item.startswith('C'))
    
    print(sorted(set_final))  # Affichage trié

# Conversion finale en liste triée
result = sorted(set_final)

print("\nListe finale triée :", result)


Qualificateur : therapy
['Cardiovascular Diseases', 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities', 'Nervous System Diseases', 'Pathological Conditions, Signs and Symptoms']

Qualificateur : pathology
['Cardiovascular Diseases', 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities', 'Nervous System Diseases', 'Pathological Conditions, Signs and Symptoms']

Qualificateur : complications
['Cardiovascular Diseases', 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities', 'Hemic and Lymphatic Diseases', 'Nervous System Diseases', 'Pathological Conditions, Signs and Symptoms', 'Skin and Connective Tissue Diseases']

Qualificateur : methods
['Cardiovascular Diseases', 'Congenital, Hereditary, and Neonatal Diseases and Abnormalities', 'Hemic and Lymphatic Diseases', 'Nervous System Diseases', 'Pathological Conditions, Signs and Symptoms', 'Skin and Connective Tissue Diseases']

Liste finale triée : ['Cardiovascular Diseases', 'Congenital, Hereditary,